<a href="https://colab.research.google.com/github/tanderson11/covid_households/blob/main/VaccineLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vaccine Study Laboratory

# Drive + Github configuration

* If you opened this notebook from Github, go to `File > Save a copy in Drive`.
* If running for the first time, set `first_time_setup = True`
* You'll have to authenticate on google drive the first time and each time the runtime times out.
* To use tokens, fork the repository and go to `Github > User > Settings > Developer Settings > Personal access tokens`

In [37]:
# A flag to clone the repository into your drive. See the >>> SETUP <<< lines to know what is affected
# Set to false after running this set of cells once
first_time_setup = True

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%mkdir /content/gdrive/My\ Drive/github
%cd /content/gdrive/My\ Drive/github

mkdir: cannot create directory ‘/content/gdrive/My Drive/github’: File exists
/content/gdrive/My Drive/github


In [3]:
# If you've forked the repository, point to your own username and repository name (if different)
repo_owner="tanderson11"
repository="covid_households"

# >>> SETUP: <<<
if first_time_setup:
    !git clone https://github.com/{repo_owner}/{repository}.git

# >>> TOKEN SETUP: <<<
# If you're using token authentication (recommended), add your token and uncomment the lines below instead

#!echo git_token=\"your_token_here\" > git_token.py
#from git_token import git_token

#!git clone https://{git_token}@github.com/{repo_owner}/{repository}

In [4]:
%cd covid_households/
!ls -a

# >>> TOKEN SETUP: <<<
# this will put your token in the right folder; comment this line out after use to avoid an error message
#!mv ../git_token.py ./

from git_token import git_token

/content/gdrive/My Drive/github/covid_households
baseline_df.hdf
comparison_df.hdf
constants.py
df.hdf
experiments
forward_simulation.py
.git
.gitignore
git_token.py
inf_var-hsar-seed_one-0.0importation-04-29-17:46
.ipynb_checkpoints
Lab.ipynb
likelihood.py
ParameterInferenceLab.ipynb
population.py
__pycache__
README.md
settings.py
SuperspreadingLab.ipynb
sus_var-hsar-seed_one-0.0importation-04-29-16:17
sus_var-hsar-seed_one-0.0importation-04-29-20:02
torch_forward_simulation.py
traits.py
utilities.py
Vaccine.ipynb
vaccine.py


# Pulling upstream changes

Uncomment this block to pull upstream changes from github

In [38]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [ ]:
# >>> PULLING CHANGES: <<<
#!git pull

# Initialization

Call this once each runtime after Drive configuration to import the module

In [5]:
# Initialization
import importlib
import vaccine
import likelihood
import utilities
import pandas as pd
import numpy as np
import functools

# Reload modules

Call this if you change any .py files in the drive

In [14]:
importlib.reload(vaccine)
importlib.reload(likelihood)
importlib.reload(utilities)

<module 'utilities' from '/content/gdrive/My Drive/github/covid_households/utilities.py'>

# Experiments

Run your experiments and interact with VaccineStudy objects within this block of cells

In [ ]:
print_documentation = True

In [ ]:
# Shows the parameters we need to create a VaccineStudy
if print_documentation:
    print(vaccine.VaccineStudy.__init__.__doc__)

In [16]:
# --- Setting parameters for a VaccineStudy ---

n_per_arm = 50 # the number of households in each arm of the study (arms are 'vaccinated' and 'placebo')
sizes = [8] # the number of households per arm are divided evenly among the sizes, if there are multiple 
#sizes = [4,5,6,7,8] 


hsar = 0.2 # the desired household sar
hsarv = 0.1 # the hsar calculated in reference to vaccinated individuals (therefore encoding vaccine effect on infectivity)

es = 0.7 # = 1 - the relative chance per contact of an infection passing to a vaccinated individual

# Duration of the study and importation rate
duration = 90 # days
importation_probability_over_study = 0.1 # probability an unvaccinateed individual becomes infected over the whole course of the study

# Specifying the vaccination protocol. Common methods such as vaccinating one individual, a fraction of individuals, or a fixed number of individuals are made available in vaccine.py
vaccination_method = vaccine.vaccinate_one
#vaccination_method = functools.partial(vaccinate_fraction,fraction_vaccinated=0.5)

In [17]:
# --- Create a study object, which provides methods for easy simulation and calculation of statistics ---
one_vax_study = vaccine.VaccineStudy("one vaccinated", n_per_arm, sizes, hsar, es, (hsarv, 'hsarv'), duration, importation_probability_over_study, vaccination_method)

In [ ]:
# What can we do with a VaccineStudy object once we have one?
if print_documentation:
    print(vaccine.VaccineStudy.__doc__)

In [ ]:
# --- Run the study some number of times ---
trials = 10
vax_df, control_df = one_vax_study.run_trials(trials, arms="both") # we collect the results of our study in two Pandas DataFrames. For more information see https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
utilities.make_bar_chart(pd.concat([vax_df, control_df])) # look at histograms of infections to check for coherence or interesting results

In [ ]:
# --- Analyze the results ---

vax_df

In [ ]:
# mean (over trials) number of placebo recepients who get infected
mean = control_df[control_df["vaccinated infected"] > 0].groupby("trialnum")["vaccinated infected"].count().mean()
total = control_df.groupby("trialnum")["num vaccinated"].count().mean()
print("Percent of placebo reccepients who were infected")
print(100 * mean / total)

# mean (over trials) number of vaccinated recepients who get infected
mean = vax_df[vax_df["vaccinated infected"] > 0].groupby("trialnum")["vaccinated infected"].count().mean() 
total = vax_df.groupby("trialnum")["num vaccinated"].count().mean()
print("Percent of vacccine reccepients who were infected")
100 * mean / total

In [ ]:
print("Mean percent of individuals infected in vaccinated households by household size")
infections = vax_df.groupby("size")["infections"].mean()
size = vax_df.groupby("size")["size"].mean()
print(infections / size * 100)

print("Mean percent of individuals infected in unvaccinated households by household size")
infections = control_df.groupby("size")["infections"].mean()
size = control_df.groupby("size")["size"].mean()
print(infections / size * 100)

In [23]:
print("VEtotal")

vetotal_df = one_vax_study.vetotal()
vetotal_df

VEtotal
Calculating VEtotal ...
 Vaccine study named one vaccinated with:
	n_per_arm                = 50
	household sizes          = {8: 50}
	es, hsarv                = 0.700, 0.100
	vax_sus, vax_inf         = 0.300, 0.463
	seeding                  = <function seed_zero at 0x7f97f12705f0>
	duration                 = 90
	net /person import prob  = 0.100
	importation rate         = 0.001
	hsar                     = 0.200
	household_beta           = 0.033
	min r0, max r0           = 1.620, 1.620



,VE,fisher p value
trialnum,,
0,0.237805,0.005118
1,0.061404,0.635258
2,-0.243478,0.041026
3,0.202797,0.033903
4,0.222973,0.015941
5,0.318584,0.003573
6,-0.051546,0.743612
7,-0.091743,0.480923
8,0.347518,0.000181


# Making a git commit from changes to python backend

To save this notebook use File > Save a copy in Github

In [28]:
!git add -A
!git status

On branch main
Your branch is behind 'origin/main' by 4 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   population.py
	modified:   utilities.py
	modified:   vaccine.py



In [29]:
!git config --global user.email ""
!git config --global user.email "Thayer"

In [30]:
message = input("Commit message? ")

!git commit -m "{message}"

Commit message? Improving documentation. Fixing imports
[main 652a385] Improving documentation. Fixing imports
 3 files changed, 18 insertions(+), 41 deletions(-)


# Pushing upstream

In [31]:
!git remote set-url origin https://{git_token}@github.com/{repo_owner}/{repository}

In [34]:
!git pull
!git commit -m 'Merged'

Already up to date.
On branch main
Your branch is ahead of 'origin/main' by 2 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [35]:
!git push

Counting objects: 8, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (8/8), done.
Writing objects: 100% (8/8), 1.12 KiB | 383.00 KiB/s, done.
Total 8 (delta 6), reused 0 (delta 0)
remote: Resolving deltas: 100% (6/6), completed with 4 local objects.
To https://github.com/tanderson11/covid_households
   f7ed928..134634f  main -> main
